## RAG(Retrieval-Augmented Generation): 검색 증강 생성
* RAG는 미리 지정한 텍스트를 데이터베이스로 준비해 두었다가 사용자가 입력하면 그 입력 내용과 연관성이 높은 텍스트를 데이터베이스에서 검색해 프롬프트에 추가해 보다 정확한 답변을 할 수 있게 하는 기법
* 질문에 더 정확하고 풍부한 답변을 주기 위해 정보 검색과 답변 생성을 결합한 기술
* 정보 검색 단계: 사용자가 질문을 하면, 외부 데이터베이스나 문서에서 관련 정보 검색
* 답변 생성 단계: 찾은 정보를 바탕으로 AI 모델이 답변 생성

# 임베딩과 벡터 DB
* 지정한 텍스트를 데이터베이스에 저장하기 위해서는 먼저 문자를 숫자로 변환하는 임베딩(embedding)이 필요하며
* 데이터를 저장하는 데이터베이스는 RDBMS가 아닌 벡터 검색에 특화된 vectorDB를 이용해야 한다.

최근 인공지능과 NLP 애플리케이션에서 **Vector Database (vectordb)**는 빠르게 인기를 얻고 있다. 이는 텍스트, 이미지, 오디오 등의 데이터를 벡터 형식으로 저장하고 검색할 수 있는 데이터베이스로, 특히 **임베딩 벡터**를 사용해 의미 기반 검색을 수행한다.

### 대표적인 Vector Database
1. **FAISS (Facebook AI Similarity Search)**:
   - **Facebook AI**에서 개발한 오픈 소스 라이브러리.
   - 매우 빠른 유사도 검색과 군집화 기능을 제공.
   - GPU 가속을 지원해 대량의 데이터를 효율적으로 처리할 수 있음.

2. **Milvus**:
   - **Zilliz**에서 개발한 오픈 소스 vectordb로, 높은 성능과 확장성을 자랑함.
   - 벡터 검색과 혼합 검색(hybrid search)을 지원해 다양한 유형의 데이터를 처리할 수 있음.
   - 분산 시스템을 통해 대규모 데이터셋에서도 효율적인 검색이 가능.

3. **Weaviate**:
   - 의미론적 검색을 위해 설계된 오픈 소스 vectordb.
   - 데이터베이스 내에서 벡터를 자동으로 생성하거나 기존의 임베딩을 사용할 수 있음.
   - 다양한 NLP 모델과의 통합을 지원.

4. **Pinecone**:
   - 클라우드 기반 벡터 데이터베이스 서비스로, 사용자가 인프라를 직접 관리할 필요 없이 벡터 검색 기능을 제공.
   - 쉽게 확장 가능하고 API 기반으로 간편하게 벡터 데이터를 관리할 수 있음.
   - 지리적으로 분산된 클러스터를 제공해 글로벌 검색 성능을 최적화함.

### Vector Database의 특징
- **의미 기반 검색**: 단순한 키워드 매칭이 아닌, 벡터 간의 거리(예: 코사인 유사도)를 이용해 데이터의 의미적 유사성을 파악.
- **빠른 검색 속도**: 수백만에서 수십억 개의 벡터에 대한 검색을 실시간으로 처리할 수 있음.
- **확장성**: 많은 vectordb는 분산 처리와 클러스터링을 통해 대규모 데이터를 효율적으로 관리할 수 있음.

### 주요 사용 사례
- **추천 시스템**: 사용자 선호도를 분석하고 유사한 제품이나 콘텐츠를 추천.
- **챗봇 및 QnA 시스템**: 의미적으로 유사한 질문과 답변을 매칭하여 더 자연스러운 대화와 검색을 가능하게 함.
- **이미지 검색**: 이미지의 특징 벡터를 사용해 시각적 유사도를 기반으로 검색.
- **문서 검색**: 임베딩 벡터를 사용하여 의미적으로 관련된 문서나 내용을 빠르게 찾음.

Vector Database는 AI와 NLP 애플리케이션의 핵심 기술로 자리잡고 있으며, 데이터의 의미를 기반으로 한 고속 검색과 처리가 필요할 때 필수적이다.


In [1]:
## 임포트
import os
import pandas as pd


In [2]:
# csv가아니므로  with로 여는점

with open("./data/hotel_data.txt", "r", encoding='utf-8' ) as f:
    data = f.read()
print(data)

# GPT가 확인해야할 내용: 호텔 CS, 운영 규칙 10가지
# 1. 호텔규정에 정확히 일치하게 안내하도록 챗봇자동화
# 2. 번호(내용)별로 분류

1. 손님 맞이
손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다.

2. 체크인과 체크아웃
체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게 체크아웃을 원하는 고객에 대해서는 객실의 공실 상황을 확인하여 가능한 한 대응해 주어야 한다. 만약 그것이 어렵다면, 짐을 일시적으로 보관할 수 있는 서비스를 제안한다.

3. Wi-Fi 및 주차장 안내
모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수 있도록 하자. 또한, 180대의 무료 주차장이 마련되어 있다. 주차장의 위치, 이용 방법, 개방 시간 등을 정확하게 안내할 수 있도록 한다.

4. 배리어 프리 대응
유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록 한다. 휠체어를 이용하는 고객이 있을 경우, 관내의 장애인 편의시설에 대해 안내하고 필요한 경우 도움을 줄 수 있도록 한다.

5. 반려동물 대응
반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을 알려주어야 한다. 이때 인근의 반려동물 동반 가능 호텔을 소개하여 고객의 불편을 덜어주어야 한다. 인근의 반려동물 호텔 정보를 항상 최신 상태로 유지해야 한다.

6. 룸 서비스
오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에 적절히 대응할 수 있도록 한다. 또한, 음식에 대한 알레르기 정보나 특별한 식단 제한에 대응할 수 있도록 주방과의 협력도 중요하다.

7. 금연 정책 및 흡연실 안내
모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실을 마련한다. 이 정보를 명확하게 전달하고, 흡연실 위치와 이용 시간을 고객에게 안내해 주어야 한다.

8. 취소 정책
취소 수

In [3]:
# # 2. 번호(내용)별로 분류의 기준은 \n\n다
data

"1. 손님 맞이\n손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다.\n\n2. 체크인과 체크아웃\n체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게 체크아웃을 원하는 고객에 대해서는 객실의 공실 상황을 확인하여 가능한 한 대응해 주어야 한다. 만약 그것이 어렵다면, 짐을 일시적으로 보관할 수 있는 서비스를 제안한다.\n\n3. Wi-Fi 및 주차장 안내\n모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수 있도록 하자. 또한, 180대의 무료 주차장이 마련되어 있다. 주차장의 위치, 이용 방법, 개방 시간 등을 정확하게 안내할 수 있도록 한다.\n\n4. 배리어 프리 대응\n유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록 한다. 휠체어를 이용하는 고객이 있을 경우, 관내의 장애인 편의시설에 대해 안내하고 필요한 경우 도움을 줄 수 있도록 한다.\n\n5. 반려동물 대응\n반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을 알려주어야 한다. 이때 인근의 반려동물 동반 가능 호텔을 소개하여 고객의 불편을 덜어주어야 한다. 인근의 반려동물 호텔 정보를 항상 최신 상태로 유지해야 한다.\n\n6. 룸 서비스\n오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에 적절히 대응할 수 있도록 한다. 또한, 음식에 대한 알레르기 정보나 특별한 식단 제한에 대응할 수 있도록 주방과의 협력도 중요하다.\n\n7. 금연 정책 및 흡연실 안내\n모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실을 마련한다. 이 정보를 명확하게 전달하고, 흡연실 위치와 이용 시간을 고객에게 안내해 주어

In [4]:
# 쪼개기
data= data.split('\n\n')
data

["1. 손님 맞이\n손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다.",
 '2. 체크인과 체크아웃\n체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게 체크아웃을 원하는 고객에 대해서는 객실의 공실 상황을 확인하여 가능한 한 대응해 주어야 한다. 만약 그것이 어렵다면, 짐을 일시적으로 보관할 수 있는 서비스를 제안한다.',
 '3. Wi-Fi 및 주차장 안내\n모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수 있도록 하자. 또한, 180대의 무료 주차장이 마련되어 있다. 주차장의 위치, 이용 방법, 개방 시간 등을 정확하게 안내할 수 있도록 한다.',
 '4. 배리어 프리 대응\n유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록 한다. 휠체어를 이용하는 고객이 있을 경우, 관내의 장애인 편의시설에 대해 안내하고 필요한 경우 도움을 줄 수 있도록 한다.',
 '5. 반려동물 대응\n반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을 알려주어야 한다. 이때 인근의 반려동물 동반 가능 호텔을 소개하여 고객의 불편을 덜어주어야 한다. 인근의 반려동물 호텔 정보를 항상 최신 상태로 유지해야 한다.',
 '6. 룸 서비스\n오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에 적절히 대응할 수 있도록 한다. 또한, 음식에 대한 알레르기 정보나 특별한 식단 제한에 대응할 수 있도록 주방과의 협력도 중요하다.',
 '7. 금연 정책 및 흡연실 안내\n모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실을 마련한다. 이 정보를 명확하게 전달하고, 흡연실 위치와 이용 시간을 고객에게

In [5]:
data[0]

"1. 손님 맞이\n손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다."

In [6]:
data[0].split('\n') # 제목과 내용사이에도 뉴라인있으니까 또 쪼개기

['1. 손님 맞이',
 "손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다."]

In [7]:
# 쪼갠 후 준비 : 딕셔너리(키밸)로 담기
for content in data:
    print(content)

1. 손님 맞이
손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다.
2. 체크인과 체크아웃
체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게 체크아웃을 원하는 고객에 대해서는 객실의 공실 상황을 확인하여 가능한 한 대응해 주어야 한다. 만약 그것이 어렵다면, 짐을 일시적으로 보관할 수 있는 서비스를 제안한다.
3. Wi-Fi 및 주차장 안내
모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수 있도록 하자. 또한, 180대의 무료 주차장이 마련되어 있다. 주차장의 위치, 이용 방법, 개방 시간 등을 정확하게 안내할 수 있도록 한다.
4. 배리어 프리 대응
유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록 한다. 휠체어를 이용하는 고객이 있을 경우, 관내의 장애인 편의시설에 대해 안내하고 필요한 경우 도움을 줄 수 있도록 한다.
5. 반려동물 대응
반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을 알려주어야 한다. 이때 인근의 반려동물 동반 가능 호텔을 소개하여 고객의 불편을 덜어주어야 한다. 인근의 반려동물 호텔 정보를 항상 최신 상태로 유지해야 한다.
6. 룸 서비스
오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에 적절히 대응할 수 있도록 한다. 또한, 음식에 대한 알레르기 정보나 특별한 식단 제한에 대응할 수 있도록 주방과의 협력도 중요하다.
7. 금연 정책 및 흡연실 안내
모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실을 마련한다. 이 정보를 명확하게 전달하고, 흡연실 위치와 이용 시간을 고객에게 안내해 주어야 한다.
8. 취소 정책
취소 수수료는 전날까

In [8]:
# content를 
for content in data:
#    print(content)
    print(content.split("\n"))

['1. 손님 맞이', "손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다."]
['2. 체크인과 체크아웃', '체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게 체크아웃을 원하는 고객에 대해서는 객실의 공실 상황을 확인하여 가능한 한 대응해 주어야 한다. 만약 그것이 어렵다면, 짐을 일시적으로 보관할 수 있는 서비스를 제안한다.']
['3. Wi-Fi 및 주차장 안내', '모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수 있도록 하자. 또한, 180대의 무료 주차장이 마련되어 있다. 주차장의 위치, 이용 방법, 개방 시간 등을 정확하게 안내할 수 있도록 한다.']
['4. 배리어 프리 대응', '유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록 한다. 휠체어를 이용하는 고객이 있을 경우, 관내의 장애인 편의시설에 대해 안내하고 필요한 경우 도움을 줄 수 있도록 한다.']
['5. 반려동물 대응', '반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을 알려주어야 한다. 이때 인근의 반려동물 동반 가능 호텔을 소개하여 고객의 불편을 덜어주어야 한다. 인근의 반려동물 호텔 정보를 항상 최신 상태로 유지해야 한다.']
['6. 룸 서비스', '오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에 적절히 대응할 수 있도록 한다. 또한, 음식에 대한 알레르기 정보나 특별한 식단 제한에 대응할 수 있도록 주방과의 협력도 중요하다.']
['7. 금연 정책 및 흡연실 안내', '모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실을 마련한다. 이 정보를 명확하게 전달하고, 흡연실 위

In [9]:
# 0번째가 제목이었고
for content in data:
#    print(content)
    print(content.split("\n")[0])
    

1. 손님 맞이
2. 체크인과 체크아웃
3. Wi-Fi 및 주차장 안내
4. 배리어 프리 대응
5. 반려동물 대응
6. 룸 서비스
7. 금연 정책 및 흡연실 안내
8. 취소 정책
9. 결제 방법
10. 항상 존중을 실천한다.


In [10]:
# 1번째가 내용
for content in data:
#    print(content)
    print(content.split("\n")[1])
    

손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다.
체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게 체크아웃을 원하는 고객에 대해서는 객실의 공실 상황을 확인하여 가능한 한 대응해 주어야 한다. 만약 그것이 어렵다면, 짐을 일시적으로 보관할 수 있는 서비스를 제안한다.
모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수 있도록 하자. 또한, 180대의 무료 주차장이 마련되어 있다. 주차장의 위치, 이용 방법, 개방 시간 등을 정확하게 안내할 수 있도록 한다.
유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록 한다. 휠체어를 이용하는 고객이 있을 경우, 관내의 장애인 편의시설에 대해 안내하고 필요한 경우 도움을 줄 수 있도록 한다.
반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을 알려주어야 한다. 이때 인근의 반려동물 동반 가능 호텔을 소개하여 고객의 불편을 덜어주어야 한다. 인근의 반려동물 호텔 정보를 항상 최신 상태로 유지해야 한다.
오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에 적절히 대응할 수 있도록 한다. 또한, 음식에 대한 알레르기 정보나 특별한 식단 제한에 대응할 수 있도록 주방과의 협력도 중요하다.
모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실을 마련한다. 이 정보를 명확하게 전달하고, 흡연실 위치와 이용 시간을 고객에게 안내해 주어야 한다.
취소 수수료는 전날까지 연락 시 숙박 요금의 30%, 당일 취소 시 50%, 연락 없이 취소할 경우 100%를 부과한다. 이 정책은 모든 예약에 적용되며, 예약 시 고객에게 이 사실을 명확히 알려야 

In [13]:
# 타이틀별 제목을 묶고 내용물끼리 묶으면 타이틀과 콘텐츠가 나눠져 분류됨
text2df={}
for content in data:
    temp=content.split("\n") # title과 콘텐츠가 같이들어감
    text2df.setdefault("title",[]).append(temp[0])  #키값이 title
    text2df.setdefault("content",[]).append(temp[1]) #키값이 content
    

In [14]:
text2df

{'title': ['1. 손님 맞이',
  '2. 체크인과 체크아웃',
  '3. Wi-Fi 및 주차장 안내',
  '4. 배리어 프리 대응',
  '5. 반려동물 대응',
  '6. 룸 서비스',
  '7. 금연 정책 및 흡연실 안내',
  '8. 취소 정책',
  '9. 결제 방법',
  '10. 항상 존중을 실천한다.'],
 'content': ["손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이 좋다. '어서 오세요' 또는 '어서 오세요' 등 상황에 맞는 표현을 사용해야 한다. 고객의 이름을 알고 있는 경우, 개인화된 인사말을 통해 고객의 만족도를 높일 수 있다.",
  '체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게 체크아웃을 원하는 고객에 대해서는 객실의 공실 상황을 확인하여 가능한 한 대응해 주어야 한다. 만약 그것이 어렵다면, 짐을 일시적으로 보관할 수 있는 서비스를 제안한다.',
  '모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수 있도록 하자. 또한, 180대의 무료 주차장이 마련되어 있다. 주차장의 위치, 이용 방법, 개방 시간 등을 정확하게 안내할 수 있도록 한다.',
  '유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록 한다. 휠체어를 이용하는 고객이 있을 경우, 관내의 장애인 편의시설에 대해 안내하고 필요한 경우 도움을 줄 수 있도록 한다.',
  '반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을 알려주어야 한다. 이때 인근의 반려동물 동반 가능 호텔을 소개하여 고객의 불편을 덜어주어야 한다. 인근의 반려동물 호텔 정보를 항상 최신 상태로 유지해야 한다.',
  '오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에 적절히 대응할 수 있도록 한다. 또한, 음식에 대한 알레르기 정보나 특별한 식단 제한에 대응할 수 있도록 주방과의

In [15]:
df=pd.DataFrame(text2df)
df

,title,content
0,1. 손님 맞이,손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이...
1,2. 체크인과 체크아웃,"체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게..."
2,3. Wi-Fi 및 주차장 안내,모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수...
3,4. 배리어 프리 대응,"유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록..."
4,5. 반려동물 대응,"반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을..."
5,6. 룸 서비스,오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에...
6,7. 금연 정책 및 흡연실 안내,모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실...
7,8. 취소 정책,"취소 수수료는 전날까지 연락 시 숙박 요금의 30%, 당일 취소 시 50%, 연락 ..."
8,9. 결제 방법,"체크아웃 시 프런트에서 현금, 신용카드, 직불카드로 결제한다. 또한 인터넷 예약을 ..."
9,10. 항상 존중을 실천한다.,"고객 한 사람 한 사람을 존중하는 태도로 대하자. 고객에 대한 예의, 배려, 전문성..."


## 텍스트 임베딩
- 라이브러리: tiktoken <br>
    - 틱토큰: OpenAI의 GPT모델에서 사용하는 토큰화 라이브러리
    - 토큰이란 LLM에서 사용하는 단위(1.5단어 기준)
    - 입력된 텍스트를 모델이 이해 할 수 있는 단위(토큰)로 분리하거나, 길이 파악(토큰 카운팅)에 쓰임
    - 이를 통해 모델의 입출력 제한을 관리함
    입출력의 토큰량에따라 비용이 달라지므로

In [16]:
!pip install tiktoken

   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   --------------------------------------- 893.9/893.9 kB 13.4 MB/s eta 0:00:00


In [19]:
import tiktoken
from openai import OpenAI

import os 
from dotenv import load_dotenv
load_dotenv("./.env_openai")
apikey = os.getenv("openai_api_key")

In [22]:
client = OpenAI()

# 임베딩 매개변수 설정
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens=1500 #임베딩할때 최대 1500 토큰까지 해달라

tokenizer=tiktoken.get_encoding(embedding_encoding)
df['n_tokens']=df.content.apply(lambda x: len(tokenizer.encode(x)))
df

,title,content,n_tokens
0,1. 손님 맞이,손님이 호텔에 도착하면 친절한 미소와 함께 예의 바르고 활기찬 인사말을 건네는 것이...,134
1,2. 체크인과 체크아웃,"체크인 시간은 오후 3시, 체크아웃 시간은 오전 11시이다. 일찍 체크인하거나 늦게...",126
2,3. Wi-Fi 및 주차장 안내,모든 객실에 무료 와이파이가 제공된다. 연결 방법과 비밀번호를 확실히 설명해 줄 수...,111
3,4. 배리어 프리 대응,"유니버설 룸의 배치와 시설, 특징을 이해하고 필요한 경우 고객에게 설명할 수 있도록...",105
4,5. 반려동물 대응,"반려동물을 동반한 고객에게는 정중하게, 그러나 분명하게 반려동물을 동반할 수 없음을...",144
5,6. 룸 서비스,오후 11시까지 룸서비스가 제공된다. 룸서비스 메뉴의 내용을 숙지하여 고객의 문의에...,120
6,7. 금연 정책 및 흡연실 안내,모든 객실은 금연입니다. 그러나 흡연자 고객의 요구를 충족시키기 위해 1층에 흡연실...,105
7,8. 취소 정책,"취소 수수료는 전날까지 연락 시 숙박 요금의 30%, 당일 취소 시 50%, 연락 ...",107
8,9. 결제 방법,"체크아웃 시 프런트에서 현금, 신용카드, 직불카드로 결제한다. 또한 인터넷 예약을 ...",108
9,10. 항상 존중을 실천한다.,"고객 한 사람 한 사람을 존중하는 태도로 대하자. 고객에 대한 예의, 배려, 전문성...",104


In [28]:
# content 임베딩해줄 함수 만들기
def get_embedding(text, model):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [29]:
df['embeddings'] =df['content'].apply(lambda x: get_embedding(x, model=embedding_model))
df

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [33]:
# csv 저장한번 해주고
df.to_csv("embeddings.csv", index=False, encoding="utf-8")

## RAG를 이용해서 GPT 만들기

In [30]:
!pip install scipy

   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   -- ------------------------------------- 2.4/41.2 MB 13.4 MB/s eta 0:00:03
   ----- ---------------------------------- 5.8/41.2 MB 14.7 MB/s eta 0:00:03
   --------- ------------------------------ 10.0/41.2 MB 16.4 MB/s eta 0:00:02
   ----------- ---------------------------- 11.5/41.2 MB 16.4 MB/s eta 0:00:02
   ------------- -------------------------- 13.6/41.2 MB 13.2 MB/s eta 0:00:03
   ---------------- ----------------------- 16.8/41.2 MB 13.4 MB/s eta 0:00:02
   ------------------- -------------------- 20.4/41.2 MB 13.9 MB/s eta 0:00:02
   ----------------------- ---------------- 23.9/41.2 MB 14.1 MB/s eta 0:00:02
   -------------------------- ------------- 27.3/41.2 MB 14.3 MB/s eta 0:00:01
   ----------------------------- ---------- 30.4/41.2 MB 14.4 MB/s eta 0:00:01
   --------------------------------- ------ 34.1/41.2 MB 14.6 MB/s eta 0:00:01
   ------------------------------------ --- 37.5/41.2 MB 14.8 M

In [31]:
from openai import OpenAI
import numpy as np
from scipy import spatial

In [32]:
client = OpenAI()

In [58]:
# 질문과 학습 데이터를 비교해서 유사도를 구하는 함수
def create_context(question, df, max_len=1800):
    # 질문을 벡터화
    q_embeddings=client.embeddings.create(input=[question], model=embedding_model).data[0].embedding
    # 질문과 RAG 비교 후 코사인 유사도 산출 후 distance 컬럼에 저장
    df['distance']=distances_from_embeddings(q_embeddings, df['embeddings'].apply(eval).apply(np.array).values, distance_metric='cosine')
    
    # 컨텍스트를 저장하기 위한 리스트
    returns =[]
    # 컨텍스트의 현재 길이
    cur_len=0
    
    # 학습데이터를 유사도 순으로 정렬하고 톸느 개수 한도까지 컨텍스트에 추가
    for _, row in df.sort_values('distances', ascending=True).iterrows():
        # 텍스트길이를 현재길이에 더하는작업
        cur_len += row['n_tokens'] + 4
        
        # 텍스트가 길어지면 종료
        if cur_len > max_len:
            break
            
        # 컨텍스트 목록에 텍스트 추가
        returns.append(row['text'])
        
    #위치 맞춰 리턴 - 하나의 문장으로 모아서 출력되게
    return "\n\n###\n\n".join(returns) 



In [68]:
## 2번째 함수
# 문맥에 따라 답해주는 기능
def answer_question(question, conversation_history):
    # file인 RAG 데이터 가져오기
    df = pd.read_csv("embeddings.csv")
    
    # 질문과 RAG 데이터를 비교해 컨텍스트 생성하기
    context = create_context(question, df, max_len=200)
    
    # 프롬프트 생성하고 대화 기록 추가하기
    prompt = "당신은 어느 호텔 직원이다. 문맥에 따라 고객의 질문에 정중하게 답하라.\n\n",
    "컨텍스트가 질문에 답할 수 없는 경우 '즉각 답을 드리기 어렵습니다. 확인이 필요한 내용입니다.'\n\n",
    f"컨텍스트: {context}\n\n---\n\n질문: {question}\n답변:",
    conversation_history.append({"role":"user", "content":prompt})
    
    try:
        # GPT에서 답변생성 
        response = client.chat.competions.create(
            model="gpt-4o-mini", 
            messages=conversation_history,
            temperature=1)
        
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e) # 오류나면 오류라고 
        return "오류"


## 문장별 토큰수를 계산해 n_tokens에 저장 시켜줄 함수
def distance_from_embeddings(querry_embdding, embeddings, distance_metric='cosine'):
    distance_metrics={
        "cosine" : spatial.distance.cosine,
        "L1" : spatioal.distance.cityblock,
        "L2" : spatioal.distance.euclidean,
        "Linf" : spatioal.distance.chebyshev,
    }
    distances = [distance_metrics[distance_metric](query_embedding, embedding) for embedding in embeddings]
    return distances




## 실행부분
함수들 import 후 

In [ ]:
conversation_history = []
while True:
    user_input = input("질문을 입력하세요. 끝내는 단어:exit")
    
    if user_input == "exit":
        break
    conversation_history.append({'role':'user', 'content':user_input})
    answer= answer_question(user_input, conversation_history)
    
    print("Chat GPT:", answer,end="\n\n")
    conversation_history.append({'role':'assistant', 'content':answer})
    
    
    
    

## FAISS VectorDB와 gradio를 이용한 RAG GPT 구현

In [45]:
!pip install faiss-cpu
# !pip install faiss-gpu #gpu 활용가능한 경우 쓰기

   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   -------- ------------------------------- 3.1/15.0 MB 20.5 MB/s eta 0:00:01
   -------------------- ------------------- 7.6/15.0 MB 18.8 MB/s eta 0:00:01
   ----------------------------- ---------- 11.0/15.0 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------  14.9/15.0 MB 19.2 MB/s eta 0:00:01
   ---------------------------------------- 15.0/15.0 MB 16.8 MB/s eta 0:00:00


In [47]:
# import 목록
import os
import pandas as pd
import numpy as np
import faiss
import gradio as gr
from openai import OpenAI
import tiktoken

C:\Users\Admin\miniforge3\envs\ai_seving\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'tiktoten'

In [49]:
# open AI  클라이언트 설정
client = OpenAI()

# 임베딩 매개변수 설정, 토크나이저까지 동일하게 작업이므로 복붙
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens=128000 #gpt-4o-mini 모델 최대 컨텍스트 길이

# 토크나이저 설정
tokenizer=tiktoken.get_encoding(embedding_encoding)


In [51]:
# 함수: 파일을 데이터프레임화
def process_file(file_path):
    with open(file_path,'r',encoding='utf-8') as f:
        data=f.read()
        
        # 복붙: 문단단위로 불러오는건 위에서 데이터프레임화 했음
    text2df={} # 호텔 가이드 데이터에 맞춰 전처리하는 코드
    for content in data:
        temp=content.split("\n") # title과 콘텐츠가 같이들어감
        text2df.setdefault("title",[]).append(temp[0])  #키값이 title
        text2df.setdefault("content",[]).append(temp[1]) #키값이 content
    df=pd. DataFrame(text2df)
    df['n_tokens'] = df['content'].apply(lambda x: len(tokenizer.encode(x)))
    return df

    

In [53]:
# 질문을 임베딩해주는 함수
def get_embedding(text, model):
    text = text.replace("\n", " ")
    #  아래가 복붙
    q_embeddings=client.embeddings.create(input=[text], model=model).data[0].embedding
    return q_embeddings

In [54]:
# 텍스트를 임베딩한 후 FAISS vector DB에 저장해주기
def save_embeddings_to_faiss(df):
    embeddings = np.array(df['embeddings'].tolist(),dtype='float32')
    index = faiss.IndexFlatL2(embeddings.shape[1]) # Euclidean distance 사용
    index.add(embeddings)
    faiss.write_index(index, "vector_index.faiss") # 파일형식으로 알아서 저장
    

In [ ]:
# 문맥을 만드는과정인가?
def create_context(question, df, max_len=128000):
    q_embeddings=client.embeddings.create(input=[text], model=embedding_model).data[0].embedding
    index = faiss.read_index("vector_index.faiss")
    D, I = index.search(np.array([q_embeddings],dtype=float32), len(df))
    
    returns = []
    cur_len=0
    for i in I[0]:
        if i == -1: # 유사도가 낮을때 제외
            continue
        row = df.iloc[i]
        cur_len += row['n_tokens'] + 4
        if cur_len > max_len:
            break
        returns.append(row['content'])
        
    return "\n\n###\n\n".join(returns)


In [80]:
## 2번째 함수 복붙
# 문맥에 따라 답해주는 기능
def answer_question(question, conversation_history):
    # file인 RAG 데이터 가져오기
    df = pd.read_csv("embeddings.csv")
    
    # 질문과 RAG 데이터를 비교해 컨텍스트 생성하기
    context = create_context(question, df, max_len=128000)
    if not context.strip(): # 컨텍스트가 없는경우
        return "죄송합니다. 질문에 대한 답을 찾을 수 없습니다"
    
    # 프롬프트 생성하고 대화 기록 추가하기
    prompt = f"다음 컨텍스트에 기반하여 질문에 답하라.\ 컨텍스트 외의 정보는 사용이 금지다.\ \n컨텍스트: {context}\n\n---\n\n질문: {question}\n답변:"
    conversation_history.append({"role":"user", "content": question})
    
    # 진행상황 표시
    print("답변을 생성 중입니다.")
    
    try:
        # GPT에서 답변생성 
        response = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[{"role":"system", "content":prompt}],
            temperature=1)
        #추가된 부분
        answer = response.choices[0].message.content.strip()
        conversation_history.append({"role":"system", "content":prompt})
        
        return answer
    except Exception as e:
        print(e) # 오류나면 오류라고 
        return "오류"




In [81]:
# 텍스트에서 데이터 프레임으로 변경한 데이터를 임베딩하는 함수
def process_and_embed(file):
    df = process_file(file.name)
    df['embeddings'] = df['content'].apply(lambda x: get_embedding(x, model=embedding_model))
    df_to_csv('embeddings.csv', index=False, encoding="utf-8")
    save_embeddings_to_faiss(df) #데이터프레임 저장
    return "임베딩 후 저장완료"

In [82]:
def chat_interface(question, history):
    answer = answer_question(question, history)
    display_history = "<br>".join([
        f"<div style='text-align: right;'>{entry['content']}</div>" if entry['role'] == 'user'
        else f"<div style='text-align: left'>{entry['content']}</div>"
        for entry in history])
                                   
    return display_history, answer

In [ ]:
# Gradio로 화면 만들기
demo = gr.Blocks()

with demo:
    gr.Markdown("# 문서 임베딩 및 질문-응답 시스템")
    with gr.Tab("파일 업로드 및 임베딩"):
        file_input = gr.File()
        embed_button = gr.Button("임베딩 및 저장")
        
        # db에 저장시키기
        embed_output = gr.Textbox()
        embed_button.click(process_and_embed, inputs=file_input, outputs=embed_output)
        
    with gr.Tab("질문 및 응답"):
        question_input = gr.Textbox(placeholder = "질문을 입력하세요")
        conversation_display = gr.HTML()
        answer_output=gr.Textbox(label="답변")
        ask_button=gr.Button("질문하기")
        
        # 버튼을 클릭했을때 위에서 만든 함수와 연결되도록
        
        ask_button.click(chat_interface, inputs=[question_input, gr.State([])],
                        outputs=[conversation_display, answer_output])

demo.launch(inline=False, server_port=7860)